# Introduction


**Gradient boosting machines (GBMs)** are currently very popular and so it's a good idea for machine learning practitioners to understand how GBMs work. The problem is that understanding all of the mathematical machinery is tricky and, unfortunately, these details are needed to tune the hyper-parameters. (Tuning the hyper-parameters is required to get a decent GBM model unlike, say, Random Forests.) My goal in this article is to explain the intuition behind gradient boosting, provide visualizations for model construction, explain the mathematics as simply as possible, and answer thorny questions such as why GBM is performing “gradient descent in function space.”

## An introduction to additive modeling

Before we get into boosting, let's look at an example of what mathematicians call additive modeling because it is the foundation of boosting. The idea is quite simple: we are going to add a bunch of simple terms together to create a more complicated expression. In the machine learning world, that expression (function) represents a model mapping some observation's feature, x, to a scalar target value, y. It's a useful technique because we can often conjure up the simple terms more easily than cracking the overall function in one go. Consider the following curve that shows y as some unknown but nontrivial function of x.

<img width="674" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/2cde6b25-856f-4340-87b7-65c9edff77e6">

Let's assume that the function is composed of several simple terms and try to guess what they are. After adding each term, we can reassess the situation to help us figure out the next term to add by considering the difference between the current combined function and the desired target function. 

Our first approximation might be the horizontal line y=30 because we can see that the y-intercept (at x=0) is 30. Check out the first plot in the following graph sequence. Naturally, the slope is wrong so we should add in the 45 degree line y=x (with slope 60/60=1) that runs through the squiggly target function, which gives us the second graph. It turns out that the squiggly bit comes from our friend the sine function so we can add that term, which leads us to the final plot matching our target function:

<img width="857" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/92a95567-9681-45e0-bf84-823a83349d02">

**Decomposing a complicated function into simpler subfunctions is nothing more than the divide and conquer strategy that we programmers use all the time. In this case, we are dividing a potentially very complicated function into smaller, more manageable bits.** For example, let's call our target function `F(x)` then we have `y = F(x) = 30 + x + sin(x)` and can abstract away the individual terms, also as functions, giving us the addition of three subfunctions:

<img width="804" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/d96772c8-9956-4fd8-b7cc-693bb728d75c">


***In the machine learning world, we're given a set of (x, y) data points rather than a continuous function, as we have here. The goal is to create a function that draws a nice curve through the data points. We call that function a model and it maps x to y, thus, making predictions given some unknown x.*** Adding up a bunch of subfunctions to create a composite function that models some data points is then called **additive modeling**. **Gradient boosting machines use additive modeling to gradually nudge an approximate model towards a really good model, by adding simple submodels to a composite model.**

## Focus on boosting

Boosting is a loosely-defined strategy that combines multiple simple models into a single composite model. The idea is that, as we introduce more simple models, the overall model becomes a stronger and **stronger predictor**. In boosting terminology, the simple models are called **weak models** or **weak learners**.

<img width="975" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/01ac4b19-1515-45ae-99d3-62c21710599b">

**Mathematicians represent both the weak and composite models as functions, but in practice the models can be anything including k-nearest-neighbors or regression trees. Since everyone uses trees for boosting, we'll focus on implementations that use regression trees for weak models,** which also happens to greatly simplify the mathematics. Later, we'll stack N feature vectors as rows in a matrix, `X = [x1, x2, ..., xN]`, and N targets into a vector, `y = [y1, y2, ..., yN]`  for N observations.

**It's often the case that an additive model can build the individual <img width="43" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/ce638cc1-8d8a-4ecc-b623-1662c4bce931">
 terms independently and in parallel, but that's not the case for boosting. Boosting constructs and adds weak models in a stage-wise fashion, one after the other, each one chosen to improve the overall model performance. The boosting strategy is greedy in the sense that choosing <img width="43" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/ce638cc1-8d8a-4ecc-b623-1662c4bce931">
 never alters previous functions. We could choose to stop adding weak models when <img width="78" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/b8c50fd0-4fcb-40be-9548-ca1320c16c06">
's performance is good enough or when <img width="43" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/ce638cc1-8d8a-4ecc-b623-1662c4bce931"> doesn't add anything. In practice, we choose the number of stages, M, as a hyper-parameter of the overall model. Allowing M to grow arbitrarily increases the risk of overfitting.** Because greedy strategies choose one weak model at a time, you will often see boosting models expressed using this equivalent, recursive formulation:

<img width="857" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/5e02fd36-cdae-4793-b72c-708d619e684a">
example, if all weak models are linear models, then the resulting meta-model is a simple linear model. If we use tiny regression trees as the weak models, the result is a forest of trees whose predictions are added together.

Let's see if we can design a strategy for picking weak models to create our own boosting algorithm for a single observation. Then, we can extend it to work on the multiple observations we'd encounter in practice.

# Gradient boosting

## The intuition behind gradient boosting 

### GBM algorithm to minimize L2 loss using the residual vector


<img width="923" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/0652af7f-8518-4aca-a1fa-edad052605d7">

<img width="928" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/11f43a6d-26ed-41c9-b934-103aa4847da0">

<img width="810" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/e0ee98ef-1ea3-4649-a7f9-a847c30a1733">


<img width="852" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/4d8935ff-0d27-4158-a588-e89f50c1777d">

<img width="885" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/a7f3d925-29ba-44ea-8330-cf7fc2658eef">


**Regression tree stumps**

See Terence's [notebook on regression tree stumps in Python](https://github.com/parrt/msds689/blob/master/notes/stumps.ipynb).

A regression tree stump is a regression tree with a single root and two children that splits on a single (feature) variable, which is what we have here, at a single threshold. (If we had more than a single value in our feature vectors, we'd have to build a taller tree that tested more variables; to avoid overfitting, we don't want very tall trees, however.) If a test feature value is less than the threshold, the model yields the average of the training target samples in the left leaf. If the test feature value is greater than or equal to the threshold, the model yields the average of the training target examples in the right leaf.

<img width="695" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/b1178ec2-6b6e-44d6-acfb-b926e60b1c4d">


<img width="988" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/f2e1633c-ff07-4fc9-9c2b-6d376449dbde">


## Measuring model performance

<img width="896" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/851707e8-a2d9-4396-a63c-ea2dcda63718">

**We show that training our Δm on the residual vector leads to a minimization of the mean squared error loss function.** First, we examine the most common form of GBM that optimizes the mean squared error (MSE), also called the L2 loss or cost. (The mean squared error is the average of the square of the difference between the true targets and the predicted values from a set of observations, such as a training or validation set.) As we'll see, A GBM is a composite model that combines the efforts of multiple weak models to create a strong model, and each additional weak model reduces the mean squared error (MSE) of the overall model. We give a fully-worked GBM example for a simple data set, complete with computations and model visualizations.


As we'll see later, we can use a different direction vector than the residual, but the basic mechanism is the same. Using the sign of the residual rather than the residual vector itself, will have the effect of minimizing a different loss function than mean squared error (it'll minimize mean absolute value). **You might've heard that gradient boosting is very complex mathematically, but that's only if we care about generalizing gradient boosting to work with any loss function (with associated direction vector), rather than the two we discuss in the first two articles of this series (residual and sign vectors).**

Also check out the next section goes through this example again **but this time training weak models on the sign of the residual not the residual vector.**

<img width="918" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/00ec6033-3c4d-48c6-b879-4ccd64241ab3">


## Choosing hyper-parameters

We've discussed two GBM hyper-parameters in this article, the **number of stages M** and the **learning rate η**.

Both affect model accuracy. The more stages we use, the more accurate the model, but the more likely we are to be overfitting. The primary value of the learning rate, or “shrinkage” as some papers call it, is to reduce overfitting of the overall model. As Chen and Guestrin say in XGBoost: A Scalable Tree Boosting System, **“shrinkage reduces the influence of each individual tree and leaves space for future trees to improve the model.”** Friedman recommends a low learning rate like 0.1 and a larger number of stages. In practice, people do a grid search over the hyper-parameter space looking for the best model performance. (Grid search can be very expensive given all of the model construction involved.) For example, see the article by Aarshay Jain: [Complete Guide to Parameter Tuning in XGBoost](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/) or the article by Jason Brownlee called [Tune Learning Rate for Gradient Boosting with XGBoost in Python](https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python).

<img width="825" alt="image" src="https://github.com/eraikakou/LLMs-News/assets/28102493/d1c0337b-464d-44e0-8ba8-63e840e37c7e">


# References

1. https://explained.ai/gradient-boosting/
1. [Gradient boosting: Distance to target](https://explained.ai/gradient-boosting/L2-loss.html)